# BI Lead Challenge

## Juan Camilo Florez Caro

Para iniciar, antes de la construcción del tablero debemos entender el objetivo del negocio; el proposito que perseguimos con el mismo. En esta etapa definiremos cuales son los KPI/ OKR que buscamos mejorar en la campaña de mercadeo, posteriormente procesaremos la data construir estos indicadores y finalmente definir el tablero que mejor se alinee con este propósito.

Entendiendo que el proposito del análisis es entender el funnel de conversión que tiene una campaña de mercadeo, la cual va desde que se intenta contactar a un cliente hasta el momento en el cual se le entrega (Bien sea de forma Fisica o digital), debemos iniciar conociendo cual es nuestra base potencial (Total de Leads).

A partir de allí encontramos que es importante definir como indicadore escenciales:

* 1) % Respuesta = # Clientes que respondieron a la campaña / Total de Leads
* 2) % Aprobación = # Clientes Aprobados / Total de clientes que llegaron a riesgo
* 3) % Entrega = # Plasticos entregados / # Plasticos Aprobados
* 4) % Uso = # Tarjetas con al menos una TX  / # Plasticos Entregados
* 5) $ Uso = Sumatoria de las transacciones sobre las cuales se generarán utilidades

Así mismo, surgen varios interrogantes como detalle del funnel anterior sobre brechas que podrian cerrar areas operativas de Mercadeo, Riesgo o Servicio al cliente revisando el detalle de la información, como lo son:

* 1) ¿Como logramos una mayor tasa de respuesta en funcion de los clientes que no nos contestaron? 
* 2) ¿Cómo logramos mejorar la tasa de aprobación ¿Cuales son los motivos por los cuales estámos negando?
* 3) ¿Cómo estan percibiendo los clientes el servicio de entrega de tarjetas fisicas?

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('BusinessBIBA_Challenge.csv', sep=',')

In [15]:
df.rename({'Unnamed: 0':'ORDER'}, axis=1, inplace=True)


Entendiendo la logica que tiene el campo STATUS, poblaremos los campos vacios con NOT RESPOND cuando tengamos un cliente que no haya respondido a la campaña de mercadeo, y TX cuando el registro contenga una Trasacción con la tarjeta


In [16]:

df['STATUS'] = np.where((df['STATUS'].isnull()==True) & (df['TXN'].isnull()==True) , 'NOT RESPOND',
                np.where((df['STATUS'].isnull()==True) & (df['TXN'].isnull()==False), 'TX',df['STATUS']))


Para mayor practicidad separamos la información correspondiente al funnel de conversion de la informacióin relacionada con las trasacciones

In [18]:

df_funnel = df.loc[:, df.columns != 'TXN'][df['STATUS']!='TX']
df_funnel.head(0)


,ORDER,ID,UPDATE,STATUS,MOTIVE,INTEREST_RATE,AMOUNT,CAT,CP,DELIVERY_SCORE


Para poder construir una metrica del total de leads que respondieron como quienes no respondieron a la campaña de maketing creamos un status denominado LEADS

In [19]:

campaign = df_funnel[(df_funnel['STATUS']=='RESPONSE')|(df_funnel['STATUS']=='NOT RESPOND')][['ID']].drop_duplicates()
campaign['STATUS'] = 'LEADS'
campaign['UPDATE'] = '00:00.0'

df_funnel = df_funnel.append(campaign)

C:\Users\1016038645\AppData\Local\Temp\ipykernel_10244\3870721461.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_funnel = df_funnel.append(campaign)


In [20]:
df_tx = df[['ORDER','ID','UPDATE','STATUS','TXN']][df['STATUS']=='TX']
df_tx.head()

,ORDER,ID,UPDATE,STATUS,TXN
3,3,1,08:23.4,TX,434.112066
10,10,4,25:47.0,TX,360.504173
11,11,4,22:22.1,TX,175.061765
12,12,4,44:53.3,TX,3719.618354
13,13,4,53:04.3,TX,1171.027491


Rectificamos que data tienen las bases ya que esta será la que llevaremos a los tableros así como también validamos cual sería el resultado de los KPI definidos inicialmente


In [31]:
total_leads = len(df_funnel['ID'].unique())
clientes_con_tx = len(df_tx['ID'].unique())
respondieron = len(df_funnel['ID'][df_funnel['STATUS']=='RESPONSE'].unique())
riesgo = len(df_funnel['ID'][df_funnel['STATUS']=='RISK'].unique())
aprobados = len(df_funnel['ID'][df_funnel['STATUS']=='APPROVED'].unique())
entregadas = len(df_funnel['ID'][df_funnel['STATUS']=='DELIVERED'].unique())
plasticos = len(df_funnel['ID'][df_funnel['MOTIVE']=='PLASTIC'].unique())
total_tx = df_tx['TXN'].sum()

print('Validacion de KPIs')
print('El dataset cuenta con ',str(total_leads), ' clientes')
print('El dataset cuenta con ',str(clientes_con_tx), ' clientes que han hecho al menos una transacción')
print('% Respuesta: ',str(respondieron/total_leads))
print('% Aprobación: ',str(aprobados/riesgo))
print('% Entrega: ',str(entregadas/plasticos))
print('% Uso: ',str(clientes_con_tx/aprobados))
print('$ Uso: ',str(total_tx))




Validacion de KPIs
El dataset cuenta con  3341  clientes
El dataset cuenta con  1255  clientes que han hecho al menos una transacción
% Respuesta:  0.6878180185573182
% Aprobación:  0.654917319408181
% Entrega:  1.0
% Uso:  0.8338870431893688
$ Uso:  7582878.185593394


In [26]:
df_funnel['STATUS'].unique()

array(['RESPONSE', 'RISK', 'APPROVED', 'NOT RESPOND', 'DELIVERED',
       'REJECTED', 'LEADS'], dtype=object)

Ahora para evaluar el servicio al cliente hemos decidido crear la metrica del NPS:


In [43]:

nps = df_funnel[df_funnel['STATUS']=='DELIVERED']
total_enc = len(nps['ID'].unique())
detractores = len(nps[nps['DELIVERY_SCORE']<=3]['ID'].unique())
pasivos = len(nps[nps['DELIVERY_SCORE']==4]['ID'].unique())
promotores = len(nps[nps['DELIVERY_SCORE']==5]['ID'].unique())

nps_sc = (promotores-detractores)/total_enc

print('El NPS es :',str(nps_sc))



El NPS es : -0.9005736137667304


Tras lo anterior nos damos cuenta que las principales acciones deberían estar encaminadas a:
1) Mejorar el % de aprobación ya que tan solo el 65% de los leads cuentan con el aval de Riesgo y siendo una campaña esta podría ser más cercana al 100% entendiendo que se preseleccionan los clientes a quienes se le hace la oferta
2) Mejorar la calidad en el servicio de la entrega de las tarjetas plásticas, ya que el NPS obtenido es del -90%

Habiendo validado los KPI que requerimos y la calidad en la data que necesitamos, exportamos las bases con las que construiremos los tableros


In [ ]:

df_funnel.to_csv('df_funnel.csv',sep='|', index= False, decimal = ',')
df_tx.to_csv('df_tx.csv',sep='|', index= False, decimal = ',')

# Construcción del tablero

Con lo anterior, entendiendo los tres tipos de Clientes Internos que podrian observar el tablero, se deciden construir 3 lienzos a tres niveles distintos:

*Lienzo Estratégico:* Un lienzo donde se observen los KPI, que no cuente con filtros para garantizar que en la organización se tenga una misma óptica sobre el desempeño de la campaña de mercadeo, un lienzo enfocado en los grandes tomadores de decisiones o C-Level

*Lienzo Táctico:* Un lienzo que permita llegar un poco más a fondo en el analisis exploratorio y descriptivo sobre la data que lo alimenta, un lienzo enfocado en los mandos medios e intermedios

*Lienzo Operativo:* Un lienzo enfocado en mostrar el detalle de los datos, enfocado en el uno a uno de aquellos puntos donde se tengan accionables que permitan movilizar los KPI inicialmente definidos. Un lienzo enfocado en quienes tienen contacto directo con el cliente

Para lo anterior el resultado obtenido lo creamos en el siguiente tablero:

LINK DEL TABLERO: https://github.com/juankfc/DataScientist-Roadmap/blob/afdc417571d5c157fa8dcfa68bad07eb3936acf7/SK%20Assessment/Credit%20Card%20Delivery%20Funnel.pbix

Donde encontraremos el siguiente resultado:

*Portada:* un menú de botones donde encontraremos los accesos cada uno de los lienzos previamente definidos

<img src="https://github.com/juankfc/DataScientist-Roadmap/blob/main/SK%20Assessment/0.png?raw=true">

*Lienzo Estrategico:* Un lienzo donde tuvimos que importar algunas visualizaciones del marketplace para construir los formatos condicionales en los KPI

<img src="https://github.com/juankfc/DataScientist-Roadmap/blob/main/SK%20Assessment/1.png?raw=true">

*Lienzo Táctico:* Un lienzo donde podemos explorar la data
<img src="https://github.com/juankfc/DataScientist-Roadmap/blob/main/SK%20Assessment/2.png?raw=true">

*Lienzo Operativo:* Un lienzo con la data accionable
<img src="https://github.com/juankfc/DataScientist-Roadmap/blob/main/SK%20Assessment/3.png?raw=true">
